## Load the houseprices data from Thinkful's database.
Run your house prices model again and assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC.
Do you think your model is satisfactory? If so, why?


In [2]:
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.diagnostic import het_breuschpagan
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


In [3]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()
drop_cols=[]
for col in df.columns:
    if df[col].isna().mean()>.4:
        drop_cols=drop_cols+[col]
df_clean=df.drop(columns=drop_cols).dropna()
df_clean['secondflrexists']=0
df_clean.loc[df['secondflrsf']>0,'secondflrexists']=1 #can prob drop 2nd fl sq ft
df_clean['totalsf']=df_clean['firstflrsf']=+df_clean['secondflrsf']
df_clean['yrsbltqared']=df_clean['yearbuilt']*df_clean['yearbuilt']
y=df_clean['saleprice']
X=df_clean.drop(columns=['saleprice','id'])
X.mszoning.dtype==int
cat_cols=[c for c in X.columns if X[c].dtype=='O']
numeric_cols=[c for c in X.columns if X[c].dtype!='O']
niceneighborhoods=['NridgHt', 'NoRidge', 'Somerst', 'Timber', 'Veenker', 'StoneBr']
df_clean['nicehood']=0
df_clean.loc[df_clean.neighborhood.isin(niceneighborhoods),'nicehood']=1
X2 = df_clean[['overallqual','grlivarea', 'garagearea', 'totalbsmtsf','yearremodadd','yearbuilt','yrsbltqared','lotarea','nicehood']]
Y2 = df_clean.saleprice
X2 = sm.add_constant(X2)
results = sm.OLS(Y2, X2).fit()
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       0.785
Model:                            OLS   Adj. R-squared:                  0.783
Method:                 Least Squares   F-statistic:                     440.4
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        22:06:20   Log-Likelihood:                -13104.
No. Observations:                1094   AIC:                         2.623e+04
Df Residuals:                    1084   BIC:                         2.628e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         -2.15e+07   5.77e+06     -3.723   

It seems satisfactory. The R2 number is decent (.785). BIC is 26280 (not sure what that means). The F-statistic is practically 0. All the variables are significant (and their coefficients make intuitive sense)

## In order to improve the goodness of fit of your model, try different model specifications by adding or removing some variables.


In [4]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1094 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               1094 non-null   int64  
 1   mssubclass       1094 non-null   int64  
 2   mszoning         1094 non-null   object 
 3   lotfrontage      1094 non-null   float64
 4   lotarea          1094 non-null   int64  
 5   street           1094 non-null   object 
 6   lotshape         1094 non-null   object 
 7   landcontour      1094 non-null   object 
 8   utilities        1094 non-null   object 
 9   lotconfig        1094 non-null   object 
 10  landslope        1094 non-null   object 
 11  neighborhood     1094 non-null   object 
 12  condition1       1094 non-null   object 
 13  condition2       1094 non-null   object 
 14  bldgtype         1094 non-null   object 
 15  housestyle       1094 non-null   object 
 16  overallqual      1094 non-null   int64  
 17  overallcond   

In [7]:
X3 = df_clean[['overallqual','grlivarea', 'garagearea', 'totalbsmtsf','yearremodadd','yearbuilt','yrsbltqared','lotarea','nicehood','fireplaces']]
Y3 = df_clean.saleprice
X3 = sm.add_constant(X3)
results3 = sm.OLS(Y3, X3).fit()
print(results3.summary())


                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       0.789
Model:                            OLS   Adj. R-squared:                  0.787
Method:                 Least Squares   F-statistic:                     404.0
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        22:14:20   Log-Likelihood:                -13095.
No. Observations:                1094   AIC:                         2.621e+04
Df Residuals:                    1083   BIC:                         2.627e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.639e+07   5.86e+06     -2.796   

Adding a fireplaces term increased R2 and decreased BIC a little bit. You can see that each fireplace increased the value by around 9000 dollars.

In [10]:
X4 = df_clean[['overallqual','grlivarea', 'garagearea', 'totalbsmtsf','yearremodadd','yearbuilt','yrsbltqared','lotarea','nicehood','fireplaces', 'wooddecksf','poolarea']]
Y4 = df_clean.saleprice
X4 = sm.add_constant(X4)
results4 = sm.OLS(Y4, X4).fit()
print(results4.summary())


                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       0.792
Model:                            OLS   Adj. R-squared:                  0.789
Method:                 Least Squares   F-statistic:                     342.2
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        22:23:15   Log-Likelihood:                -13087.
No. Observations:                1094   AIC:                         2.620e+04
Df Residuals:                    1081   BIC:                         2.627e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.606e+07   5.85e+06     -2.748   

Adding pool area and deck sf somewhat improve our R2 but not the BIC.

In [11]:
df_clean['CentralAC']=0
df_clean.loc[df_clean['centralair']=='Y','CentralAC']=1
X4 = df_clean[['overallqual','grlivarea', 'garagearea', 'totalbsmtsf','yearremodadd','yearbuilt','yrsbltqared','lotarea','nicehood','fireplaces','CentralAC']]
Y4 = df_clean.saleprice
X4 = sm.add_constant(X4)
results4 = sm.OLS(Y4, X4).fit()
print(results4.summary())


                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       0.789
Model:                            OLS   Adj. R-squared:                  0.787
Method:                 Least Squares   F-statistic:                     368.2
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        22:29:09   Log-Likelihood:                -13094.
No. Observations:                1094   AIC:                         2.621e+04
Df Residuals:                    1082   BIC:                         2.627e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.408e+07   6.01e+06     -2.341   

Adding a central AC term doesn't improve the R2 at all.

The best one might be the one that includes fireplaces, wood deck space, and pool area. I don't think any of those were particularly multicolinear, so that isn't much of a problem, plus it explains the most variance.